In [9]:
# In your xArm serial module (adapt from docs)
import xarm

# arm is the first xArm detected which is connected to USB
arm = xarm.Controller('USB')
print('Battery voltage in volts:', arm.getBatteryVoltage())
states = {
        'home': [90, 90, 90, 90, 90, 90],  # Central safe pos (calibrate once)
        'ready_to_grab': [45, 120, 90, 90, 135, 90],  # Above fixed piece spot, gripper open/down
        'ready_to_move': [45, 120, 135, 90, 90, 90]   # Lifted 5-10cm, gripper closed
    }
def set_state(state_name):

    if state_name in states:
        arm.setPosition(states[state_name])  # Your method: send via TTL serial
        print(f"Set to {state_name}")
    else:
        print("Invalid state")

def get_joints():
    return arm.read_positions()  # Returns list of 6 floats + voltages

def check_grip_success(threshold=0.5):  # Via end-effector feedback
    positions, voltages = arm.read_positions_and_voltages()
    # Simple: Grip success if voltage spike on gripper servo (j6?)
    return any(v > threshold for v in voltages[-1:])

set_state('ready_to_grab')


Battery voltage in volts: 7.523


AttributeError: 'Controller' object has no attribute 'set_positions'